In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
df = pd.read_csv('./data/totalwithmaininfo-re-3.csv',sep=',')
df = df.drop(['respondent'],axis=1)

In [3]:
df

,EAR,MAR,Circularity,MOE,Y,EAR_N,MAR_N,Circularity_N,MOE_N
0,0.094047,0.249984,0.252997,2.658085,1,-13.710813,-6.632470,-5.465475,188.365026
1,0.123201,0.377198,0.260764,3.061654,1,-11.318740,1.364749,-5.167676,246.986888
2,0.120386,0.310688,0.282237,2.580770,1,-11.549712,-2.816360,-4.344351,177.134403
3,0.260234,0.571277,0.296206,2.195246,1,-0.075324,13.565338,-3.808766,121.133736
4,0.176730,0.197190,0.296562,1.115770,1,-6.926761,-9.951340,-3.795122,-35.669328
...,...,...,...,...,...,...,...,...,...
17995,0.380395,0.594847,0.575300,1.563762,0,1.705177,10.130346,2.534295,7.990808
17996,0.395783,0.434963,0.580511,1.098994,0,2.421697,1.648345,2.689264,-1.593234
17997,0.383986,0.403942,0.583252,1.051970,0,1.872395,0.002658,2.770767,-2.562906
17998,0.383426,0.431997,0.583910,1.126678,0,1.846308,1.491029,2.790331,-1.022349


In [4]:
train_percentage = 21/25
train_index = int(len(df)*train_percentage)
test_index = len(df)-train_index

In [5]:
df_train = df[:train_index]
df_test = df[-test_index:]

In [6]:
X_test = df_test.drop(["Y"],axis=1)
y_test = df_test["Y"]

In [7]:
X_train = df_train.drop('Y',axis=1)
y_train = df_train['Y']

In [8]:
def average(y_pred):
  for i in range(len(y_pred)):
    if i % 240 == 0 or (i+1) % 240 == 0:
      pass
    else: 
      average = float(y_pred[i-1] +  y_pred[i] + y_pred[i+1])/3
      if average >= 0.5:
        y_pred[i] = 1
      else:
        y_pred[i] = 0
  return y_pred

In [9]:
clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)
print(clf)
y_pred_1 = clf.predict(X_test)
# y_pred_1 = average(y_pred_1)
# y_score_1 = clf.predict_proba(X_test)[:,1]
acc1 = accuracy_score(y_test, y_pred_1)
# f1_score_1 = metrics.f1_score(y_test, y_pred_1)
# roc_1 = metrics.roc_auc_score(y_test, y_score_1)
# print([acc1,f1_score_1,roc_1])
# print(confusion_matrix(y_test, y_pred_1))

LogisticRegression(max_iter=1000)


In [10]:
print(acc1)

0.9454861111111111


In [11]:
from sklearn.tree import DecisionTreeClassifier
acc5=[]
max_depth = []
for i in [2,3,4,5,6,7,8,9,10]:
    clf_DT = DecisionTreeClassifier(random_state=0, max_depth = i)
    clf_DT.fit(X_train, y_train)
    pred_DT = clf_DT.predict(X_test)
    pred_DT = average(pred_DT)
    acc5.append(accuracy_score(pred_DT, y_test))
    max_depth.append(i)
print (max(acc5))

0.9913194444444444


In [12]:
from sklearn.ensemble import RandomForestClassifier
acc6=[]
max_depth = []
for i in range(1,10):
    clf_RF = RandomForestClassifier(max_depth=i)
    clf_RF.fit(X_train, y_train) 
    pred_RF = clf_RF.predict(X_test)
    pred_RF = average(pred_RF)
    acc6.append(accuracy_score(pred_RF, y_test))
    max_depth.append(i)
print (max(acc6))

0.9913194444444444


In [13]:
acc3_list = []
f1_score3_list = []
roc_3_list = []
from sklearn.neighbors import KNeighborsClassifier
for i in range(1,30):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train) 
    pred_KN = neigh.predict(X_test)
    pred_KN = average(pred_KN)
    y_score_3 = neigh.predict_proba(X_test)[:,1]
    acc3_list.append(accuracy_score(y_test, pred_KN))
    f1_score3_list.append(metrics.f1_score(y_test, pred_KN))
    roc_3_list.append(metrics.roc_auc_score(y_test, y_score_3))

In [14]:
acc3_list.index(max(acc3_list))+1

2

In [15]:
neigh = KNeighborsClassifier(n_neighbors=acc3_list.index(max(acc3_list))+1)
neigh.fit(X_train, y_train) 
pred_KN = neigh.predict(X_test)
pred_KN = average(pred_KN)
y_score_3 = neigh.predict_proba(X_test)[:,1]
acc3 = accuracy_score(y_test, pred_KN)
f1_score_3 = metrics.f1_score(y_test, pred_KN)
roc_3 = metrics.roc_auc_score(y_test, y_score_3)
print([acc3,f1_score_3,roc_3])
print(confusion_matrix(y_test, pred_KN))

[0.8368055555555556, 0.6109271523178809, 0.8115786448818736]
[[2041  324]
 [ 146  369]]


In [16]:
#KNN

acc3_list = []
f1_score3_list = []
roc_3_list = []
from sklearn.neighbors import KNeighborsClassifier
for i in range(1,30):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train) 
    pred_KN = neigh.predict(X_test)
    pred_KN = average(pred_KN)
    y_score_3 = neigh.predict_proba(X_test)[:,1]
    acc3_list.append(accuracy_score(y_test, pred_KN))
    f1_score3_list.append(metrics.f1_score(y_test, pred_KN))
    roc_3_list.append(metrics.roc_auc_score(y_test, y_score_3))
    
    
neigh = KNeighborsClassifier(n_neighbors=acc3_list.index(max(acc3_list))+1)
neigh.fit(X_train, y_train) 
pred_KN = neigh.predict(X_test)
pred_KN = average(pred_KN)
y_score_3 = neigh.predict_proba(X_test)[:,1]
acc3 = accuracy_score(y_test, pred_KN)
# f1_score_3 = metrics.f1_score(y_test, pred_KN)
# roc_3 = metrics.roc_auc_score(y_test, y_score_3)
print([acc3])
print(confusion_matrix(y_test, pred_KN))

[0.8368055555555556]
[[2041  324]
 [ 146  369]]
